# Day 9: Rope Bridge ---



This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..
...##.
.####.
....#.
s###..
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

# Approach

First up, this seems really interesting.

Initial thoughts:
1. What data structure to hold H and T positions? Do I just hold their coordinates?
2. Need answer in form of positions visited... so it's a boolean 2D array, set to True when tail lands on it, sum up the trues to get the count.
3. Do I need to be able to print one of these diagrams? Probably not. But it would be cool.

4. Where does it start? How large is the grid? These aren't specified, so may need to dynamically resize the grid as I go
5. Does that change my mind on how to store the positions visited... maybe a set of serialized coordinates would be a better approach.


## How does the tail move?
Logic is: --WRONG--
    H moved in a direction D
    If the "distance" (could include diagonal distance of 1) between H and T is more than 1:
        Move T in direction D

I think I'm going to go a little bit OO in my approach today.

"Distance" doesn't actually need calcualting, just need a boolean answer to "is it too far away"

Why is this wrong... if on a diagonal, when the head pulls away, the tail snaps back in directly behind.

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....


So I need to revisit my move logic!

    H moved in a direction D
    If the "distance" (could include diagonal distance of 1) between H and T is more than 1:
        Move T directly "behind" H, where "behind" is opposite direction of move.



Hmm... it works for the test data, but if failing for real input.

Re-reading the puzzle:
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up

I believe this is the same as my logic.



In [202]:
class Rope:
    
    def __init__(self, numKnots=2):
        self.numKnots = numKnots
        self.knotPos:[tuple[int,int]] = []
        for _ in range(numKnots):
            self.knotPos.append((0,0))
        
        self.tailpositions:set[tuple[int, int]] = set()
        self.recordTailPosition()

    def recordTailPosition(self):
        #print(self.knotPos[-1])
        self.tailpositions.add(self.knotPos[-1])

    def positionsVisited(self):
        return len(self.tailpositions)

    #move head eith U,D,L or R
    def move(self,direction: str):
        headVector = (0,0)
        match direction:
            case 'U':
                headVector = (0,1)
            case 'D':
                headVector = (0,-1)
            case 'L':
                headVector = (-1,0)
            case 'R':
                headVector = (1,0)
            case _:
                raise Exception('Invalid direction')
        
        #update each knot in turn
        for i in range(self.numKnots):
            if i == 0:
                #move the head
                head = self.knotPos[0]
                self.knotPos[0] = (head[0]+headVector[0], head[1]+headVector[1])
            else:
                #each subsequent knot
                head = self.knotPos[i-1]
                tail = self.knotPos[i]
                dx = head[0] - tail[0]
                dy = head[1] - tail[1]

                #does it need to move? 
                if abs(dx) > 1 or abs(dy) > 1:
                    #but they only move 1 sq diagonally... so neither dx or dy should be greater than 1
                    if dx!=0:
                        dx = int(dx / abs(dx))
                    if dy!=0:
                        dy = int(dy / abs(dy))
                
                    self.knotPos[i] = (tail[0] + dx, tail[1] + dy)            


    def processMoveInstructions(self, input:str):
        for l in input.splitlines():
            #print(l)
            direction, num = l.split(' ')
            #print('Direction: ' + direction + ' // Num: ' + str(int(num)))
            for i in range(int(num)):
                self.move(direction)
                self.recordTailPosition()
            #print(self)




In [203]:
#Tests
testData = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""

testRope = Rope()
testRope.processMoveInstructions(testData)
# print(testRope)
print(len(testRope.tailpositions))
#moves should be 13


13


In [204]:
#real input
input = open("day9input.txt").read()
myrope = Rope()
myrope.processMoveInstructions(input)
print(len(myrope.tailpositions))


6391


FRUSTRATION... my answer is 6402 - too high. Why?



HOURS LATER.....


Grr.. python class syntax is annoying. If you declare property outside __init__ then it's global state. \
So my list of tail positions wasn't getting cleared when I created a new instance of Rope. FML. That's hours of my life I'm never getting back.


# Part 2

Longer rope.. 

In [207]:
part2testdata = """R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20"""

part2numberofknots = 10

part2testanswer = 36

part2rope = Rope(part2numberofknots)
part2rope.processMoveInstructions(part2testdata)
print(part2rope.positionsVisited())
print(part2rope.positionsVisited()==part2testanswer)

36
True


In [208]:
#real run for Part 2
p2r = Rope(part2numberofknots)
p2r.processMoveInstructions(input)
print(p2r.positionsVisited())

2593
